In [1]:
import re
import json
import importlib
import numpy as np
import pandas as pd
from collections import deque
import utils

In [2]:
importlib.reload(utils)

<module 'utils' from '/home/fei/Documents/projects/lyrics/encoder-decoder/utils.py'>

In [3]:
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [4]:
encoder_model = load_model('encoder_model.h5')

/home/fei/miniconda3/envs/py3_5/lib/python3.5/site-packages/keras/models.py:251: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [5]:
decoder_model = load_model('decoder_model.h5')

/home/fei/miniconda3/envs/py3_5/lib/python3.5/site-packages/keras/models.py:251: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [6]:
word2ind, ind2word = utils.load_index_word_map()

In [7]:
n_tokens = len(word2ind)

In [26]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, n_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, word2ind[' ']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.random.choice(list(range(n_tokens)), p=output_tokens[0, -1, :])
        sampled_char = ind2word[str(sampled_token_index)]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > 40):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, n_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [19]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    target_seq = np.array(word2ind[' ']).reshape(1, -1)
    stop_condition = False
    decoded_sentence = []
    decoded_indices = []
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        # Sample a token
        sampled_token_index = np.random.choice(list(range(vocab_size)), p=output_tokens[0, -1, :])
        decoded_indices.append(sampled_token_index)
        sampled_char = ind2word[str(sampled_token_index)]
        decoded_sentence.append(sampled_char)

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > 25):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.array(sampled_token_index).reshape(1, -1)

        # Update states
        states_value = [h, c]

    return decoded_sentence, decoded_indices

In [10]:
encoder_input_data = pd.read_csv('encoder_input.csv').values

In [11]:
def make_3d_one_hot(input_2d):
    z_coords = input_2d.flatten()
    y_mesh, x_mesh = np.meshgrid(list(range(input_2d.shape[1])), list(range(input_2d.shape[0])))
    x_coords = x_mesh.flatten()
    y_coords = y_mesh.flatten()
    input_data = np.zeros((*input_2d.shape, n_tokens))
    input_data[x_coords, y_coords, z_coords] = 1
    return input_data

In [12]:
encoder_input_data = make_3d_one_hot(encoder_input_data)

In [27]:
decode_sequence(encoder_input_data[:1, :])

'  a ttalue n o    2  ne              \n'